# Imports

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# American Option Pricing

## Simulate Paths

In [2]:
def simulate_gbm_paths(S0, r, sigma, T, M, N):
    """
    Simule des trajectoires GBM (Euler discret)
    M = nombre de pas de temps
    N = nombre de trajectoires
    """
    dt = T / M
    Z = np.random.normal(size=(N, M))
    S = np.zeros((N, M+1))
    S[:, 0] = S0

    for t in range(M):
        S[:, t+1] = S[:, t] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t])
        
    return S

## Longstaff Schwarz Algorithm

In [ ]:
def longstaff_schwarz(S, K, r, T, option_type="put"):
    """
    Implémentation Longstaff-Schwarz pour option américaine
    S : trajectoires shape (N, M+1)
    """
    N, M_plus_1 = S.shape
    M = M_plus_1 - 1
    dt = T / M
    
    # Payoff immédiat
    if option_type == "put":
        payoff = np.maximum(K - S, 0)
    else:
        payoff = np.maximum(S - K, 0)

    # Cashflows initiaux (terminal payoff)
    cashflow = payoff[:, -1].copy()

    # Backward induction
    for t in range(M-1, 0, -1):

        # ITM paths
        itm = payoff[:, t] > 0
        S_itm = S[itm, t]
        cf_itm = cashflow[itm] * np.exp(-r * dt)

        # Regression: base polynômiale (ordre 2 recommandé)
        if len(S_itm) > 0:
            X = np.vstack([np.ones(len(S_itm)), S_itm, S_itm**2]).T
            beta = np.linalg.lstsq(X, cf_itm, rcond=None)[0]
            
            continuation_value = X @ beta
        else:
            continuation_value = np.zeros_like(S_itm)

        # Exercice optimal
        exercise = payoff[itm, t] > continuation_value
        
        # Mettre à jour cashflows
        new_cf = cashflow.copy()
        new_cf[itm] = np.where(exercise, payoff[itm, t], cashflow[itm] * np.exp(-r * dt))

        # Si exercice : bloquer la suite
        exercised_paths = itm.copy()
        exercised_paths[itm] = exercise
        new_cf[exercised_paths] = payoff[exercised_paths, t]

        cashflow = new_cf

    # Prix = moyenne actualisée
    price = np.mean(cashflow * np.exp(-r * dt))

    return price

## Test

In [5]:
if __name__ == "__main__":
    S0 = 100
    K = 110
    r = 0.05
    sigma = 0.2
    T = 1.0
    M = 50   # pas de temps
    N = 50_000  # trajectoires

    S = simulate_gbm_paths(S0, r, sigma, T, M, N)
    price = longstaff_schwarz(S, K, r, T, "call")

    print("Prix de l’option américaine :", price)

Prix de l’option américaine : 6.186614095203198
